# Item 07. 결과 부족이 발생할 경우 null과 Failure를 사용하라

## 코틀린은 CheckedException이 존재하지 않는다.

- CheckedException은 코드를 더 복잡하게 만들고 개발자의 불필요한 수고를 늘린다는 전제로 코틀린은 자바와 달리 CheckedException이 존재하지 않음
- 하지만 비즈니스 로직을 작성하다보면 특정 메서드는 커스텀한 비즈니스 예외를 던질것이라고 알려줘야하고, 이 함수를 사용하는 개발자도 이를 알아야할 필요성이 존재하는 경우가 많다.
- 이때, @Throws 어노테이션으로 알릴수도 있지만, 이방식은 문서화에 가까우면 컴파일러 차원에서 도움을 받지 못함

> 코틀린쓰면서 강제성있는 에러핸들링이 뭔가 아쉽다고 느껴짐!

## 예외가 정보를 전달하거나 코드의 제어흐름을 관리하는데 쓰이는것은 바람직하지 않다.

- 예외가 전파되는 과정을 추적하기는 쉽지 않음
- 예외는 컴파일러 최적화를 방해함
- 비즈니스의 제어를 할때는 예외보다는 null이나 Result 객체를 사용하는것이 좀더 바람직
- 예외적인 상황에만 예외를 던지도록하자

## 소결: 사용자의 입장에서 신경써야할 코드보다는 사용자입장의 코드까지 작성해 노출시키는것이 더 바람직


In [0]:
import java.lang.IllegalArgumentException

// 앞으로 이 코드를 사용하는 사용자는 항상 예외 핸들링을 신경써야함!
class Service {
    fun process(input: String) {
        require(input.isNotBlank()){"invalid input!"}
        println("do process")
    }
}

fun clientCode() {
    val service = Service()
    runCatching {
        service.process("")
    }.getOrElse {
        throw IllegalArgumentException("fail")
    }
}

clientCode()

In [0]:
// Result로 반환한다면?
class Service {
    fun process(input: String): Result<Unit> {
        require(input.isNotBlank()) { "invalid input!" }
        println("do process")
    }
}

fun clientCode() {
    val service = Service()
    service.process("")
        .onSuccess { it }
        .onFailure { throw it }
}

clientCode()

- Result 객체나 Null을 사용하면 최소한 비정상적인 결과값이 나올수 있다는 api 스펙을 CheckedException 처럼 간접적으로 알릴수 있음
- 해당 코드를 사용하는 입장에서 좀더 안전한 사용이 가능

하지만 정확히 어떤 에러를 던질지에 대한 컴파일러 체크는 불가능...


## 실무적으로 써먹을만한 컨벤션을 고민하자면...?

비즈니스 로직의 단위인 usecase는 반드시 Result를 반환하고, 해당 유즈케이스에 쓰이는, 혹은 해당 도메인에 쓰이는 상위 비즈니스 예외를 when으로 핸들링하게끔 컨벤션을 하면 Checked Ex같은 효과를 낼수는 있다.

하지만 이도 결국 해당 메서드에서 나올 정확한 예외들을 컴파일러가 가르키는방식은 아니고 간접적인 방식이라는 한계가 존재...

그래도 유즈케이스의 리턴을 무조건 Result로 내보내는것은 고려해볼만하다.

사용하는 어댑터쪽에서는 fold로 항상 에러핸들링하기!
